https://nbviewer.jupyter.org/github/natasha/yargy/blob/master/docs/ref.ipynb#%D0%9F%D1%80%D0%B5%D0%B4%D0%B8%D0%BA%D0%B0%D1%82%D1%8B

In [1]:
import json
import pandas as pd
from transliterate  import translit
from yargy.tokenizer import MorphTokenizer
import pymorphy2

In [46]:
data = pd.read_json('bread-probe.json')
data

,index,id,category_id,name,description,images,manufacturer,stock,prices,prices_per_unit
0,0,202797,412331,"Тортилья FINE LIFE пшеничная, 250 г",,[https://cdn.metro-cc.ru/ru/ru_pim_53788300100...,"{'id': 1965, 'name': 'FINE LIFE', 'image': None}","{'value': 144, 'text': 'Товара мало'}","{'price': 80.89, 'type': 'discount_levels', 'd...","{'price': 80.89, 'type': 'discount_levels', 'd..."
1,1,220111,412331,"Тортилья пшеничная 25 см Metro Chef, 858 г х 1...",Тортилья пшеничная METRO CHEF изготавливается ...,[https://cdn.metro-cc.ru/ru/ru_pim_18462300100...,"{'id': 18524, 'name': 'METRO CHEF', 'image': N...","{'value': 260, 'text': 'Товара достаточно'}","{'price': 140, 'type': 'discount_levels', 'dis...","{'price': 140, 'type': 'discount_levels', 'dis..."
2,2,140720,412331,"Тортилья пшеничная Metro Chef, 12х30 см","Тортилья пшеничная Metro Chef, 12х30 см – плос...",[https://cdn.metro-cc.ru/ru/ru_pim_18462500100...,"{'id': 18524, 'name': 'METRO CHEF', 'image': N...","{'value': 127, 'text': 'Товара мало'}","{'price': 177, 'type': 'discount_levels', 'dis...","{'price': 177, 'type': 'discount_levels', 'dis..."
3,3,206483,412331,"Тортилья пшеничная 20 см 8шт Metro Chef, 296г","Тортилья пшеничная 20 см 8шт Metro Chef, 296 г...",[https://cdn.metro-cc.ru/ru/ru_pim_18462000100...,"{'id': 18524, 'name': 'METRO CHEF', 'image': N...","{'value': 280, 'text': 'Товара много'}","{'price': 69.99, 'type': 'discount_levels', 'd...","{'price': 69.99, 'type': 'discount_levels', 'd..."
4,4,154296,412331,"Тортилья пшеничная с отрубями FINE LIFE, 286г",Польша,[https://cdn.metro-cc.ru/ru/ru_pim_53789100100...,"{'id': 1965, 'name': 'FINE LIFE', 'image': None}","{'value': 49, 'text': 'Заканчивается'}","{'price': 96.59, 'type': 'discount_levels', 'd...","{'price': 96.59, 'type': 'discount_levels', 'd..."
5,5,191447,412331,"Тортилья шпинатная FINE LIFE, 25см, 286г",Лепешка тортилья FINE LIFE в ассортименте,[https://cdn.metro-cc.ru/ru/ru_pim_53789000100...,"{'id': 1965, 'name': 'FINE LIFE', 'image': None}","{'value': 195, 'text': 'Товара достаточно'}","{'price': 94.09, 'type': 'discount_levels', 'd...","{'price': 94.09, 'type': 'discount_levels', 'd..."
6,6,517275,412331,"Хлеб DR'SCHAR белый пан бланко, 250г",Хлеб белый DR. SCHAR PAN BLANCO предназначен д...,[https://cdn.metro-cc.ru/ru/ru_pim_53260700100...,"{'id': 22061, 'name': 'DR'SCHAR', 'image': None}","{'value': 35, 'text': 'Заканчивается'}","{'price': 277, 'type': 'discount_levels', 'dis...","{'price': 277, 'type': 'discount_levels', 'dis..."
7,7,477917,412331,"Хлеб сэндвич DAN CAKE xxl пшеничный в нарезке,...",Мягкий хлеб от бренда Dan Cake полностью готов...,[https://cdn.metro-cc.ru/ru/ru_pim_47835000100...,"{'id': 16442, 'name': 'DAN CAKE', 'image': None}","{'value': 17, 'text': 'Заканчивается'}","{'price': 201, 'type': 'discount_levels', 'dis...","{'price': 201, 'type': 'discount_levels', 'dis..."
8,8,517397,412331,"Хлеб DR'SCHAR Pan Multigrano зерновой, 250 г",Хлеб DR. SCHAR PAN MULTIGRANO – зерновой низко...,[https://cdn.metro-cc.ru/ru/ru_pim_53261100100...,"{'id': 22061, 'name': 'DR'SCHAR', 'image': None}","{'value': 48, 'text': 'Заканчивается'}","{'price': 276, 'type': 'discount_levels', 'dis...","{'price': 276, 'type': 'discount_levels', 'dis..."
9,9,457070,412331,"Сухари FINE LIFE с изюмом, 350 г",,[https://cdn.metro-cc.ru/ru/ru_pim_44576300100...,"{'id': 1965, 'name': 'FINE LIFE', 'image': None}","{'value': 131, 'text': 'Товара мало'}","{'price': 66.9, 'type': 'discount_levels', 'di...","{'price': 66.9, 'type': 'discount_levels', 'di..."


**Непосредственно алгоритм**

Проверка данных

In [47]:
lines = data['name'].head(5).tolist()
lines

['Тортилья FINE LIFE пшеничная, 250 г',
 'Тортилья пшеничная 25 см Metro Chef, 858 г х 12 шт',
 'Тортилья пшеничная Metro Chef, 12х30 см',
 'Тортилья пшеничная 20 см 8шт Metro Chef, 296г',
 'Тортилья пшеничная с отрубями FINE LIFE, 286г']

Закгрузка библиотек

In [48]:
from yargy import (
    Parser,
    rule, or_, and_, not_,
)

from yargy.predicates import (
    caseless, type, gram, normalized,
    in_, in_caseless, dictionary
)

from yargy.pipelines import (
    caseless_pipeline,
    morph_pipeline
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy import interpretation as interp


def test(rule, *tests):
    parser = Parser(rule)
    for line, etalon in tests:
        match = parser.match(line)
        assert match, line
        guess = match.fact
        assert etalon == guess, guess

Получение списка производителей

In [49]:
vendorlist = []
for i in range(0,len(data['manufacturer'])):
    item = data['manufacturer'][i]['name'];
    if(item not in vendorlist):
        vendorlist.append(item)
        alt = translit(item, 'ru');
        if(alt != item):
            vendorlist.append(alt)
vendorlist

['FINE LIFE',
 'ФИНЕ ЛИФЕ',
 'METRO CHEF',
 'МЕТРО ЦХЕФ',
 "DR'SCHAR",
 'ДРЬСЦХАР',
 'DAN CAKE',
 'ДАН ЦАКЕ',
 'DELICADOS',
 'ДЕЛИЦАДОС',
 'BEZGLUTEN',
 'БЕЗГЛУТЕН',
 'GERBER',
 'ГЕРБЕР',
 'ARO',
 'АРО',
 'ХЛЕБНЫЙ ДОМ',
 "HARRY'S",
 'ХАРРЫЬС',
 'FAZER',
 'ФАЗЕР',
 'ЛИДЕР',
 'ЕВРОХЛЕБ',
 'РИЖСКИЙ ХЛЕБ']

Модель

In [50]:
Item = fact(
    'item',
    ['type', 'vendor']
)

In [51]:
TYPE = morph_pipeline(['молоко'])

In [8]:
VENDOR = morph_pipeline(vendorlist)

In [9]:
ITEM = rule(
    TYPE.interpretation(
        Item.type
    ),
    VENDOR.interpretation(
        Item.vendor
    ).optional()
).interpretation(
    Item
)

In [36]:
text = ''' 3 бутылки молока валио, 4 бутылки молока фрутоняня '''
parser = Parser(ITEM)
match = parser.find(text)
display(match)
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

Match(
    tokens=[MorphToken(
         value='молока',
         span=[11, 17),
         type='RU',
         forms=[Form('молоко', Grams(NOUN,Sgtm,gent,inan,neut,sing)),
          Form('молока', Grams(NOUN,femn,inan,nomn,sing))]
     ),
     MorphToken(
         value='валио',
         span=[18, 23),
         type='RU',
         forms=[Form('валио', Grams(Fixd,NOUN,Name,Sgtm,anim,masc,nomn,sing)),
          Form('валио', Grams(Fixd,NOUN,Name,Sgtm,anim,gent,masc,sing)),
          Form('валио', Grams(Fixd,NOUN,Name,Sgtm,anim,datv,masc,sing)),
          Form('валио', Grams(Fixd,NOUN,Name,Sgtm,accs,anim,masc,sing)),
          Form('валио', Grams(Fixd,NOUN,Name,Sgtm,ablt,anim,masc,sing)),
          Form('валио', Grams(Fixd,NOUN,Name,Sgtm,anim,loct,masc,sing)),
          Form('валио', Grams(ADJF,Fixd,Qual,Subx,masc,nomn,sing)),
          Form('валио', Grams(ADJF,Fixd,Qual,Subx,gent,masc,sing)),
          Form('валио', Grams(ADJF,Fixd,Qual,Subx,datv,masc,sing)),
          Form('валио', Grams(

['молока', 'валио']
['молока', 'фрутоняня']


In [11]:
dir(parser)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'chart',
 'complete',
 'extract',
 'find',
 'findall',
 'match',
 'matches',
 'predict',
 'rule',
 'scan',
 'tagger',
 'tokenizer']

In [52]:
def findWordOccurance(data):
    occ = {}
    for name in data['name'].values:
        words = name.split()
        for word in words:
            word = word.upper().strip()
            if not word in occ:
                occ[word] = 0
            occ[word] = occ[word]+1
    sorted_dict = {}
    sorted_keys = sorted(occ, key=occ.get,reverse=True)

    for key in sorted_keys:
        sorted_dict[key] = occ[key]
    return sorted_dict

In [53]:
names = findWordOccurance(data)
names

{'Г': 17,
 'ХЛЕБ': 14,
 'FINE': 11,
 'С': 9,
 'ТОРТИЛЬЯ': 8,
 'METRO': 7,
 'CHEF,': 7,
 'ТАРТАЛЕТКИ': 7,
 'LIFE,': 6,
 'LIFE': 5,
 'ШТ': 5,
 '500Г': 5,
 'ПШЕНИЧНАЯ': 4,
 '250Г': 4,
 'СУХАРИ': 4,
 '350': 4,
 'ДЛЯ': 4,
 'ИЗ': 4,
 'СЛОЕНОГО': 4,
 'СМ': 3,
 'DELICADOS': 3,
 'БЕЗ': 3,
 'ХЛЕБНЫЙ': 3,
 'ДОМ': 3,
 'ЗАВИТКИ': 3,
 '250': 2,
 '286Г': 2,
 "DR'SCHAR": 2,
 'ПШЕНИЧНЫЙ': 2,
 'ЗЕРНОВОЙ,': 2,
 'ВАНИЛЬНЫЕ,': 2,
 'ПЕСОЧНЫЕ': 2,
 '27': 2,
 '400Г': 2,
 'ИКРЫ': 2,
 '600': 2,
 'ТЕСТА': 2,
 'ДЕРЕВЕНСКИЙ': 2,
 'ГЛЮТЕНА,': 2,
 'МАКОМ': 2,
 'НАРЕЗКА,': 2,
 'ТЕСТА,': 2,
 'ГЕРКУЛЕС': 2,
 'ПШЕНИЧНАЯ,': 1,
 '25': 1,
 '858': 1,
 'Х': 1,
 '12': 1,
 '12Х30': 1,
 '20': 1,
 '8ШТ': 1,
 '296Г': 1,
 'ОТРУБЯМИ': 1,
 'ШПИНАТНАЯ': 1,
 '25СМ,': 1,
 'БЕЛЫЙ': 1,
 'ПАН': 1,
 'БЛАНКО,': 1,
 'СЭНДВИЧ': 1,
 'DAN': 1,
 'CAKE': 1,
 'XXL': 1,
 'В': 1,
 'НАРЕЗКЕ,': 1,
 '750Г': 1,
 'PAN': 1,
 'MULTIGRANO': 1,
 'ИЗЮМОМ,': 1,
 'САЛАТА': 1,
 '560': 1,
 'СЫРОМ,': 1,
 '80': 1,
 'БОЛЬШИЕ': 1,
 '450': 1,
 'ТЕМНЫЙ': 1,
 '260': 1,


In [54]:
def normalizeKeys(orgKeys,vendorlist=[]):
    morph = pymorphy2.MorphAnalyzer()
    tokenizer = MorphTokenizer()
    for key in list(orgKeys):
        keyData = list(tokenizer(key))
        p = morph.parse(key)[0]
    
        if(key in vendorlist):
            orgKeys.remove(key)
            continue
        if(len(key) <= 1):
            orgKeys.remove(key)
            continue
        if(len(keyData)>1):
            orgKeys.remove(key)
            continue
        if({'UNKN'} in p.tag):
            orgKeys.remove(key)
            continue
        if({'NUMB'} in p.tag):
            orgKeys.remove(key)
            continue
    return orgKeys

In [55]:
keys = list((names.keys()))
keysNormalized = normalizeKeys(keys,vendorlist)

In [56]:
keysNormalized

['ХЛЕБ',
 'FINE',
 'ТОРТИЛЬЯ',
 'METRO',
 'ТАРТАЛЕТКИ',
 'LIFE',
 'ПШЕНИЧНАЯ',
 'СУХАРИ',
 'ДЛЯ',
 'ИЗ',
 'СЛОЕНОГО',
 'СМ',
 'БЕЗ',
 'ХЛЕБНЫЙ',
 'ДОМ',
 'ЗАВИТКИ',
 'ПШЕНИЧНЫЙ',
 'ПЕСОЧНЫЕ',
 'ИКРЫ',
 'ТЕСТА',
 'ДЕРЕВЕНСКИЙ',
 'МАКОМ',
 'ГЕРКУЛЕС',
 'ОТРУБЯМИ',
 'ШПИНАТНАЯ',
 'БЕЛЫЙ',
 'ПАН',
 'СЭНДВИЧ',
 'DAN',
 'CAKE',
 'XXL',
 'PAN',
 'MULTIGRANO',
 'САЛАТА',
 'БОЛЬШИЕ',
 'ТЕМНЫЙ',
 'БАТОНЧИК',
 'БАРАНКИ',
 'ТОСТОВЫЙ',
 'ЛЕПЕШКИ',
 'TORTILLAS',
 'СМЕСЬ',
 'ВЫПЕЧКИ',
 'ХЛЕБА',
 'КГ',
 'САХАРА',
 'КОРИЦЕЙ',
 'САХАРОМ',
 'AMERICAN',
 'SANDWICH',
 'ЭНЕРГИЯ',
 'ЗДОРОВЬЯ',
 'СТРОЙНЫЙ',
 'ХЛЕБЦЫ',
 'ЗЕРНОВОЙ',
 'ЕВРОПЕЙСКИЙ',
 'СВЕТЛЫЙ',
 'РИЖСКИЙ',
 'КРЕСТЬЯНСКИЙ',
 'БЕЗДРОЖЖЕВОЙ',
 'ЗАВАРНОЙ']

Получаем имя категории

In [57]:
categoryName = list(keysNormalized)[0]
categoryName

'ХЛЕБ'

Все остальное - под тип

In [58]:
keysNormalized.remove(categoryName)
keysNormalized

['FINE',
 'ТОРТИЛЬЯ',
 'METRO',
 'ТАРТАЛЕТКИ',
 'LIFE',
 'ПШЕНИЧНАЯ',
 'СУХАРИ',
 'ДЛЯ',
 'ИЗ',
 'СЛОЕНОГО',
 'СМ',
 'БЕЗ',
 'ХЛЕБНЫЙ',
 'ДОМ',
 'ЗАВИТКИ',
 'ПШЕНИЧНЫЙ',
 'ПЕСОЧНЫЕ',
 'ИКРЫ',
 'ТЕСТА',
 'ДЕРЕВЕНСКИЙ',
 'МАКОМ',
 'ГЕРКУЛЕС',
 'ОТРУБЯМИ',
 'ШПИНАТНАЯ',
 'БЕЛЫЙ',
 'ПАН',
 'СЭНДВИЧ',
 'DAN',
 'CAKE',
 'XXL',
 'PAN',
 'MULTIGRANO',
 'САЛАТА',
 'БОЛЬШИЕ',
 'ТЕМНЫЙ',
 'БАТОНЧИК',
 'БАРАНКИ',
 'ТОСТОВЫЙ',
 'ЛЕПЕШКИ',
 'TORTILLAS',
 'СМЕСЬ',
 'ВЫПЕЧКИ',
 'ХЛЕБА',
 'КГ',
 'САХАРА',
 'КОРИЦЕЙ',
 'САХАРОМ',
 'AMERICAN',
 'SANDWICH',
 'ЭНЕРГИЯ',
 'ЗДОРОВЬЯ',
 'СТРОЙНЫЙ',
 'ХЛЕБЦЫ',
 'ЗЕРНОВОЙ',
 'ЕВРОПЕЙСКИЙ',
 'СВЕТЛЫЙ',
 'РИЖСКИЙ',
 'КРЕСТЬЯНСКИЙ',
 'БЕЗДРОЖЖЕВОЙ',
 'ЗАВАРНОЙ']

Пееревод в транслит

In [59]:
def translitNames(names):
    for name in names:
        alt = translit(name, 'ru');
        if(alt != name):
            names.append(alt)
    return names

In [60]:
translitNames(keysNormalized)

['FINE',
 'ТОРТИЛЬЯ',
 'METRO',
 'ТАРТАЛЕТКИ',
 'LIFE',
 'ПШЕНИЧНАЯ',
 'СУХАРИ',
 'ДЛЯ',
 'ИЗ',
 'СЛОЕНОГО',
 'СМ',
 'БЕЗ',
 'ХЛЕБНЫЙ',
 'ДОМ',
 'ЗАВИТКИ',
 'ПШЕНИЧНЫЙ',
 'ПЕСОЧНЫЕ',
 'ИКРЫ',
 'ТЕСТА',
 'ДЕРЕВЕНСКИЙ',
 'МАКОМ',
 'ГЕРКУЛЕС',
 'ОТРУБЯМИ',
 'ШПИНАТНАЯ',
 'БЕЛЫЙ',
 'ПАН',
 'СЭНДВИЧ',
 'DAN',
 'CAKE',
 'XXL',
 'PAN',
 'MULTIGRANO',
 'САЛАТА',
 'БОЛЬШИЕ',
 'ТЕМНЫЙ',
 'БАТОНЧИК',
 'БАРАНКИ',
 'ТОСТОВЫЙ',
 'ЛЕПЕШКИ',
 'TORTILLAS',
 'СМЕСЬ',
 'ВЫПЕЧКИ',
 'ХЛЕБА',
 'КГ',
 'САХАРА',
 'КОРИЦЕЙ',
 'САХАРОМ',
 'AMERICAN',
 'SANDWICH',
 'ЭНЕРГИЯ',
 'ЗДОРОВЬЯ',
 'СТРОЙНЫЙ',
 'ХЛЕБЦЫ',
 'ЗЕРНОВОЙ',
 'ЕВРОПЕЙСКИЙ',
 'СВЕТЛЫЙ',
 'РИЖСКИЙ',
 'КРЕСТЬЯНСКИЙ',
 'БЕЗДРОЖЖЕВОЙ',
 'ЗАВАРНОЙ',
 'ФИНЕ',
 'МЕТРО',
 'ЛИФЕ',
 'ДАН',
 'ЦАКЕ',
 'XXЛ',
 'ПАН',
 'МУЛТИГРАНО',
 'ТОРТИЛЛАС',
 'АМЕРИЦАН',
 'САНДWИЦХ']

Алгоритм поиска

In [64]:
vendorlist

['FINE LIFE',
 'ФИНЕ ЛИФЕ',
 'METRO CHEF',
 'МЕТРО ЦХЕФ',
 "DR'SCHAR",
 'ДРЬСЦХАР',
 'DAN CAKE',
 'ДАН ЦАКЕ',
 'DELICADOS',
 'ДЕЛИЦАДОС',
 'BEZGLUTEN',
 'БЕЗГЛУТЕН',
 'GERBER',
 'ГЕРБЕР',
 'ARO',
 'АРО',
 'ХЛЕБНЫЙ ДОМ',
 "HARRY'S",
 'ХАРРЫЬС',
 'FAZER',
 'ФАЗЕР',
 'ЛИДЕР',
 'ЕВРОХЛЕБ',
 'РИЖСКИЙ ХЛЕБ']

In [93]:
INT = type('INT')
Item = fact(
    'item',
    ['amount','type', 'vendor']
)

TYPE = morph_pipeline([categoryName])
VENDOR = morph_pipeline(vendorlist)

ITEM_Category = rule(
    TYPE.interpretation(
        Item.type
    )
).interpretation(
    Item
)

ITEM_Vendor = rule(
    VENDOR.interpretation(
        Item.vendor
    )
).interpretation(
    Item
)


ITEM_DIGIT = INT.interpretation(
    interp.custom(int)
)

text = ''' 3 буханки Еврохлеба, 4 хлеба Гербер '''

parser = Parser(ITEM_Category)
match = parser.find(text)
display(match)
for match in parser.findall(text):
    print([_.value for _ in match.tokens])
          
parser = Parser(ITEM_Vendor)
match = parser.find(text)
display(match)
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

parser = Parser(ITEM_DIGIT)
match = parser.find(text)
display(match)
for match in parser.findall(text):
    print([_.value for _ in match.tokens])
    print()

Match(
    tokens=[MorphToken(
         value='хлеба',
         span=[24, 29),
         type='RU',
         forms=[Form('хлеб', Grams(NOUN,gent,inan,masc,sing)),
          Form('хлеб', Grams(NOUN,inan,masc,nomn,plur)),
          Form('хлеб', Grams(NOUN,accs,inan,masc,plur))]
     )],
    span=[24, 29)
)

['хлеба']


Match(
    tokens=[MorphToken(
         value='Еврохлеба',
         span=[11, 20),
         type='RU',
         forms=[Form('еврохлеб', Grams(NOUN,gent,inan,masc,sing)),
          Form('еврохлеб', Grams(NOUN,inan,masc,nomn,plur)),
          Form('еврохлеб', Grams(NOUN,accs,inan,masc,plur))]
     )],
    span=[11, 20)
)

['Еврохлеба']
['Гербер']


Match(
    tokens=[Token(
         value='3',
         span=[1, 2),
         type='INT'
     )],
    span=[1, 2)
)

['3']

['4']



Tree(
    root=Node(
        rule=BNFRule(
            productions=[Production(
                 terms=[TypePredicate(
                      value='INT'
                  )],
                 main=0
             )],
            name='R0',
            interpretator=NormalizerInterpretator(
                normalizer=FunctionNormalizer(
                    function=int
                )
            ),
            relation=None
        ),
        production=Production(
            terms=[TypePredicate(
                 value='INT'
             )],
            main=0
        ),
        rank=0,
        children=[Leaf(
             predicate=TypePredicate(
                 value='INT'
             ),
             token=Token(
                 value='3',
                 span=[1, 2),
                 type='INT'
             )
         )]
    ),
    range=(0,
     1)
)